# HuggingFace smolagents: Lightweight and Focused Agents

**Objective:** This notebook introduces `smolagents`, a lightweight and easy-to-understand agent framework from HuggingFace. You will learn its core philosophy and how to build a simple, tool-using agent based on the official documentation.

**Target Audience:** Software engineers attending the AI-Driven Software Engineering Program.

**Core Philosophy:** Simplicity and clarity. `smolagents` is not designed to be a sprawling, all-encompassing framework. Instead, it provides a minimal, clean, and effective implementation of a tool-using agent. It's an excellent choice for learning the fundamentals of the ReAct (Reason+Act) loop or for projects that need a simple, focused agent without a lot of overhead.

## 1. Setup

We will install `smolagents` and its dependencies. As per the official documentation, we'll include the `[litellm]` extra, which is required to use OpenAI models like GPT-4o.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    print("ERROR: OPENAI_API_KEY not found. Please check your .env file.")

## 2. Foundational Agent with a Custom Tool

The primary use case for `smolagents` is creating agents that can use tools. To ensure compatibility, we must define our custom tools as classes that inherit from the library's `Tool` base class and implement a `forward` method for the execution logic.

The process involves:
1.  Defining a custom tool class that inherits from `smolagents.tools.Tool`.
2.  Initializing the `CodeAgent` with a `LiteLLMModel` wrapper and an instance of our custom tool class.
3.  Running the agent with a prompt.

In [3]:
from smolagents import CodeAgent, LiteLLMModel
from smolagents.tools import Tool
from typing import ClassVar

# 1. Define a custom tool class inheriting from smolagents' Tool
class GetCompanyFoundingYearTool(Tool):
    """A custom tool to get the founding year of a tech company."""
    name: ClassVar[str] = "get_company_founding_year"
    description: ClassVar[str] = "Returns the founding year of a major tech company."
    inputs: ClassVar[dict] = {
        "company_name": {
            "type": "string",
            "description": "The name of the company to look up."
        }
    }
    output_type: ClassVar[str] = "string"

    def forward(self, company_name: str) -> str:
        company_name = company_name.lower()
        if "google" in company_name:
            return "1998"
        elif "microsoft" in company_name:
            return "1975"
        elif "booz allen hamilton" in company_name:
            return "1914"
        else:
            return "I don't know that company's founding year."

# 2. Initialize the Model using the LiteLLMModel wrapper for OpenAI models
model = LiteLLMModel(model_id="gpt-4o")

# 3. Initialize the Agent with an instance of our custom tool
smol_agent = CodeAgent(
    model=model, 
    tools=[GetCompanyFoundingYearTool()]
)

# 4. Run the agent
print("--- Running smol-agent ---")
# The .run() method will print the verbose thought process by default.
response = smol_agent.run("What year was Booz Allen Hamilton founded?")

print(f"\n--- Final Answer ---")
print(response)

/Users/agaleana/repos/AG-AISOFTDEV/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- Running smol-agent ---


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What year was Booz Allen Hamilton founded?                                                                      │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  founding_year = get_company_founding_year(company_name="Booz Allen Hamilton")                                    
  final_answer(founding_year)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 1914

[Step 1: Duration 1.83 seconds| Input tokens: 1,935 | Output tokens: 55]


--- Final Answer ---
1914


## 3. Advanced Capability: Iterative Refinement (Code Generation)

`smolagents` can also be used for iterative tasks like code generation and refinement. We can simulate a developer providing feedback to the agent to improve its code.

In [4]:
# For this example, we'll create an agent with no tools, focusing on its reasoning.
model = LiteLLMModel(model_id="gpt-4o")

code_gen_agent = CodeAgent(
    model=model,
    tools=[]
)

print("--- Code Generation: First Pass ---")
# The system prompt / persona is now part of the task given to the .run() method
initial_code = code_gen_agent.run("You are an expert Python programmer. Write a simple Python function to add two numbers.")
print(f"\n--- Generated Code ---\n{initial_code}")

print("\n--- Code Refinement: Second Pass ---")
# We can continue the conversation by passing the previous response as context.
refined_code = code_gen_agent.run(
    f"Here is the function you wrote:\n\n{initial_code}\n\nPlease refine it by adding type hints and a Google-style docstring."
)
print(f"\n--- Refined Code ---\n{refined_code}")

--- Code Generation: First Pass ---


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an expert Python programmer. Write a simple Python function to add two numbers.                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  def add_two_numbers(a, b):                                                                                       
      """Returns the sum of two numbers."""                                                                        
      return a + b                                                                                                 
                                                                                                                   
  # Example usage of the function                                                                                  
  result = add_two_numbers(5, 3)                                                                                   
  print(f"The sum of 5 and 3 is: {result}")                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The sum of 5 and 3 is: 8

Out: None

...

Out: <function create_function.<locals>.new_func at 0x122647ba0>

[Step 20: Duration 3.88 seconds| Input tokens: 93,359 | Output tokens: 2,652]

Reached max steps.

[Step 21: Duration 5.80 seconds| Input tokens: 99,248 | Output tokens: 2,800]


--- Refined Code ---
Certainly! Here's the refined version of the `add_two_numbers` function, complete with type hints and a Google-style docstring:

```python
def add_two_numbers(a: float, b: float) -> float:
    """Adds two numbers and returns their sum.

    Args:
        a (float): The first number to add.
        b (float): The second number to add.

    Returns:
        float: The sum of the two numbers.
    """
    return a + b
```

This version of the function now clearly indicates the expected input types and return type using type annotations, and includes a docstring that follows the Google Python style guide, explaining the function's purpose, its parameters, and its return value.


## Lab Conclusion

This lab has provided a concise introduction to `smolagents`. You've learned how to create a simple, tool-using agent and how to use the agent for iterative refinement tasks.

**Key Takeaways:**
- `smolagents` is a great choice for projects that require a simple, lightweight, and easy-to-understand agent.
- It provides a clear implementation of the ReAct loop for tool use.
- Creating custom tools by inheriting from the library's `Tool` class and implementing a `forward` method with a matching signature is the correct way to ensure compatibility.